# Things to be solved..
- pt no. 109(no FLAIR) & 123(no T1w) & 709(no FLAIR) should be excluded!
- choice :  
1. Normalization(for 2D slice? or 3D bulk image?) : 2 options
2. Again Normlization(for Patch? or for overall 2D(or 3D) image?) : * 2 options
3. Normalization method (mean-norm, zero-mean, unit-varia~) : * n options
#### 그런데! But all related to normalization.. (학습 속도를 빠르게 하느냐 차이니까...뭐가 되었던 normalization을 해주기만 하면 되는거 아닌지?)

## 0. Load required libraries & csv files

In [ ]:
# Install required packages
! pip install natsort

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt

import os 
import cv2
import numpy as np
import glob
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

#from torch.utils.data import Dataset, DataLoader
import SimpleITK as sitk

# to sort file names by its order
from natsort import natsorted
%matplotlib inline

# Global variables
modalities = ['FLAIR', 'T1w', 'T1wCE', 'T2w']

In [ ]:
# Load y : labels & preds
train_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
preds_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
#train_labels = pd.read_csv('../png_data/train_labels.csv')
#preds_labels = pd.read_csv('../png_data/sample_submission.csv')

print(f'Number of patients = {len(train_labels)}'.format())
train_labels.head()

## 1. Show sample

In [ ]:
# Reference: https://www.kaggle.com/ayuraj/brain-tumor-eda-and-interactive-viz-with-w-b
def ReadMRI(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    # min-max normalization
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
def load_imgs(idx, train=True): 
    images_idx = {}
    for modal in modalities:
        images_modal = []
        if train:
            file_path_list = glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'+idx+'/'+modal+'/*')
        else:
            file_path_list = glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'+idx+'/'+modal+'/*')
            
        # Should be sorted again. 
        file_path_list = natsorted(file_path_list)
        
        for file_path in file_path_list:
            image = ReadMRI(file_path)
            # In case, direc is empty!
            if len(image) == 0:
                print('yes')
                image = np.zeros((1,256,256)) # pt no. 109(no FLAIR) & 123(no T1w) & 709(no FLAIR) >> excluded! 
            images_modal.append(image)
        images_idx[modal] = np.array(images_modal)
        
    return images_idx

In [ ]:
%%time
# Just to check how long would "load_imgs" be. 
# %%time 위에 주석 달면 time 체크 불가....!?
for i in range(1):
    idx = str(train_labels.BraTS21ID[i]).zfill(5)
    print(idx)
    imgs = load_imgs(idx)
    img_ = imgs[modalities[0]]#.mean(axis=0)
    print(img_.shape)

In [ ]:
# Again, just to check sample
fig, ax = plt.subplots(2,2, figsize=(10,10))
for i in range(2):
    for j in range(2):
        m = ax[i,j].imshow(imgs[modalities[2*i+j]].mean(axis=0))
        ax[i,j].set_title(modalities[2*i+j])
plt.show()

# 2. Preprocessing each image (per pt, modality)

In [ ]:
# 1. Create a data loader with N4biasFieldCorrectionImageFilter
class PreprocessedImage2Dver():
    def __init__(self, list_BraTS21ID, list_labels=None,
                dim=(512, 512), n_modals=len(modalities), n_classes=2,
                 num_slices_from_center:int=3,
                 is_train=True, transform = None): # For single pt. 
        self.dim = dim
        ### ????? 1. How to do batch norm for this task..?
        #self.batch_size = batch_size
        self.list_labels = list_labels
        self.is_train = (list_labels is not None)
        self.list_BraTS21ID = list_BraTS21ID
        self.n_modals = n_modals # number of modalities
        self.num_slices_from_center = num_slices_from_center
        
    
    def __getitem__(self, index):
        BraTS21ID_temp = self.list_BraTS21ID[index] #self.list_BraTS21ID[index*self.batch_size:(index+1)*self.batch_size] # index로 적어준 batch 만 진행!
        
        X = self.__data_generation(BraTS21ID_temp)
        
        if self.is_train:
            y = self.list_labels[index] #self.list_labels[index*self.batch_size:(index+1)*self.batch_size]
            return np.array(X), np.array(y)
        else:
            return np.array(X)
    
    def __data_generation(self, BraTS21ID_temp):
        new_imgs = np.zeros((self.num_slices_from_center*2,  *self.dim, self.n_modals))
        #print(new_imgs.shape)
        #new_imgs = None
        
        idx = str(BraTS21ID_temp).zfill(5)
        imgs = load_imgs(idx, train=self.is_train) # imgs = {'FLAIR': ~, 'T1w': ~, 'T1wCE': ~} lib다. 
        
        index_modal = 0
        for modal in modalities:
            corrct_norm_imgs_modal = []

            img_modal = imgs[modal] #.shape :  ex. (288, 256, 256) 
            
            if img_modal.shape[0] < self.num_slices_from_center *2 :
                print('The number of slice is smaller than total number of slices!!')
                break
            
            central_slices = int(img_modal.shape[0]/2)
            
            list_slices = list(range(central_slices-self.num_slices_from_center, central_slices+self.num_slices_from_center))
            for slice_i in list_slices:
                img_slice_i = cv2.resize(img_modal[slice_i,:,:], dsize= self.dim, interpolation = cv2.INTER_LINEAR) #???
                img_slice_i_ndarray = np.array(img_slice_i, dtype = 'float32')

                # 1. Removing radiofrequency inhomogeneity using N4BiasFieldCorrection
                # ref : https://www.kaggle.com/josepc/rsna-effnet
                inputImage = sitk.GetImageFromArray(img_slice_i_ndarray)
                maskImage = sitk.GetImageFromArray((img_slice_i_ndarray>0.1)*1) #sitk.OtsuThreshold(inputImage, 0,1,200) 
                inputImage = sitk.Cast(inputImage, sitk.sitkFloat32) #?? 왜 32로?
                maskImage = sitk.Cast(maskImage, sitk.sitkUInt8) #?? 왜 8로?
                corrector = sitk.N4BiasFieldCorrectionImageFilter()
                numberFittingLevels = 4 # ??
                maxIter = 100 # ??
                if maxIter is not None:
                    corrector.SetMaximumNumberOfIterations([maxIter]*numberFittingLevels) # ??
                corrected_image = corrector.Execute(inputImage, maskImage)
                
                corrected_image = sitk.GetArrayFromImage(corrected_image)
                
                # 2. Normalization (근데 앞에 load_imgs에서 normalization term이 있긴 했음.)
                # ????? 2. 학습 속도만 해결하면 되는거니, 어떻게든 normalization만 하면 되는걸지?
                max_2D = np.amax(corrected_image) 
                min_2D = np.amin(corrected_image)
                mean_2D = np.mean(corrected_image)
                
                normalized_corrected_image = (corrected_image-min_2D)/(max_2D-min_2D)
                
                
                corrct_norm_imgs_modal.append(normalized_corrected_image)
                
            corrct_norm_imgs_modal = np.array(corrct_norm_imgs_modal)
            
            new_imgs[:, :, :, index_modal] = corrct_norm_imgs_modal
            
            index_modal += 1
        
        return new_imgs # shape = (num_slices_from_center*2, *dim, n_modals) !!! not 3D. 4D.

## 3. Check sample preprocessed image
### - num_slices_from_center:int=3
### - for "pt no.00002" 
### - for 2D slice image at center

In [ ]:
# Do train_test_split. 
# - Train : Test = 8:2 / Stratified random sampling / random_state = 42
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_labels.BraTS21ID, train_labels.MGMT_value,
                                                 test_size = .2, random_state = 42, stratify = train_labels.MGMT_value)

In [ ]:
# ??????? 3. 원래의 W*H 사이즈를 보존해야?
dim = (256, 256) 
train_set = PreprocessedImage2Dver(X_train, y_train, dim=dim)
val_set = PreprocessedImage2Dver(X_val, y_val, dim=dim)
test_set = PreprocessedImage2Dver(preds_labels.BraTS21ID,  dim = dim)

In [ ]:
sample_preprocessed_imgs = train_set[1][0] # pt no.00002

center_slice_idx = int(sample_preprocessed_imgs.shape[0]/2)
#print(center_slice_idx) # In this cas, returns 3. 
img_at_center = sample_preprocessed_imgs[center_slice_idx] # slice at center

In [ ]:
# Check sampled space of sample_img (FLAIR ver). 
print(img_at_center[100:150, 100:130, 0])

In [ ]:
# Again, just to check sample preprocessed image.
fig, ax = plt.subplots(2,2, figsize=(10,10))
for i in range(2):
    for j in range(2):
        m = ax[i,j].imshow(img_at_center[:, :, 2*i+j], vmin=0, vmax=1)
        ax[i,j].set_title(modalities[2*i+j])
plt.show()

### Before preprocessing

In [ ]:
sample_pt_ID = str(train_labels.BraTS21ID[1]).zfill(5)

before_prepro_img = load_imgs(idx=sample_pt_ID)

In [ ]:
print('Before preprocessing - pt no.00002')
# Again, just to check sample
fig, ax = plt.subplots(2,2, figsize=(10,10))
for i in range(2):
    for j in range(2):
        center_slice_idx = int(before_prepro_img[modalities[2*i+j]].shape[0]/2)
        #print(center_slice_idx)
        m = ax[i,j].imshow(before_prepro_img[modalities[2*i+j]][center_slice_idx])
        ax[i,j].set_title(modalities[2*i+j])
plt.show()